In [1]:
# 🔧 0) 모듈 경로 추가 (Jupyter/VSCode 노트북/스크립트 공통)
import sys, os
# 프로젝트 루트(kiwoom 폴더)에서 실행한다고 가정: ./src 경로 추가
sys.path.insert(0, os.path.join(os.getcwd(), "src"))

# 확인용 (선택)
print(sys.path[:3])

['c:\\Users\\edukd\\OneDrive\\문서\\GitHub\\kiwoom\\src', 'c:\\Users\\edukd\\miniforge3\\envs\\kiwoombank\\python311.zip', 'c:\\Users\\edukd\\miniforge3\\envs\\kiwoombank\\DLLs']


In [2]:
# ✅ 1) DART 키 초기화
from kiwoom_finance.dart_client import init_dart
init_dart()  # .env 또는 환경변수에서 로드

# ✅ 2) 기업 로딩 → 전처리
from kiwoom_finance.dart_client import find_corp, extract_fs
from kiwoom_finance.preprocess import preprocess_all

corp = find_corp("005380")
fs = extract_fs(corp, bgn_de="20170101", report_tp="annual", separate=False)
bs_flat, is_flat, cis_flat, cf_flat = preprocess_all(fs)

# ✅ 3) 진단 스니펫 (현재 가지고 계신 그대로)
# ... (당신이 붙여넣은 코드 계속)

Error occurred during getting browser(s): random, but was suppressed with fallback.


In [17]:
# ✅ 0) DART 키 초기화 (옵션 A: .env에 DART_API_KEY가 있으면 이 줄만으로 충분)
from kiwoom_finance.dart_client import init_dart
init_dart()  # .env 또는 환경변수에서 자동 로드

# (옵션 B: 키를 직접 넘기고 싶으면 아래 줄 사용)
# init_dart(api_key="여기에_본인_DART_API_KEY")

# ✅ 1) 기업 선택 → 재무제표 로딩 → 전처리
from kiwoom_finance.dart_client import find_corp, extract_fs
from kiwoom_finance.preprocess import preprocess_all

corp = find_corp("005380")  # 현대차 예시. 다른 종목코드/회사명으로 바꿔도 됨
fs = extract_fs(corp, bgn_de="20170101", report_tp="annual", separate=False)
bs_flat, is_flat, cis_flat, cf_flat = preprocess_all(fs)

print("✔ corp:", corp.corp_name, corp.stock_code)
print("✔ bs_flat shape:", bs_flat.shape)
print("✔ bs_flat latest index:", max(map(int, bs_flat.index)) if len(bs_flat.index)>0 else None)

# ✅ 2) BS 컬럼 alias 매핑 확인 + 최신 날짜로 주요 지표 직접 계산
import pandas as pd

CA_CANDS  = ["유동자산"]
CL_CANDS  = ["유동부채","유동부채총계","단기부채","유동부채합계"]
TL_CANDS  = ["부채총계","부채 총계"]
EQ_CANDS  = ["자본총계","자본 총계"]
INV_CANDS = ["재고자산","상품","제품","반제품","원재료","재공품","저장품"]

def pick_first(df, cands):
    for c in cands:
        if c in df.columns:
            return c
    return None

ca_col  = pick_first(bs_flat, CA_CANDS)
cl_col  = pick_first(bs_flat, CL_CANDS)
tl_col  = pick_first(bs_flat, TL_CANDS)
eq_col  = pick_first(bs_flat, EQ_CANDS)
inv_col = pick_first(bs_flat, INV_CANDS)

print("🔎 MAPPED COLS:", {"CA": ca_col, "CL": cl_col, "TL": tl_col, "EQ": eq_col, "INV": inv_col})
print("🔎 BS columns (first 25):", bs_flat.columns.tolist()[:25])

if ca_col and cl_col:
    latest_idx = int(sorted(bs_flat.index)[-1])  # BS 기준 최신 날짜
    row = bs_flat.loc[latest_idx]

    to_num = lambda x: pd.to_numeric(str(x).replace(",", ""), errors="coerce")

    ca  = to_num(row.get(ca_col))
    cl  = to_num(row.get(cl_col))
    tl  = to_num(row.get(tl_col)) if tl_col else pd.NA
    eq  = to_num(row.get(eq_col)) if eq_col else pd.NA
    inv = to_num(row.get(inv_col)) if inv_col else 0  # 재고 없으면 0으로 가정

    current_ratio = (ca / cl) if pd.notna(ca) and pd.notna(cl) and cl != 0 else pd.NA
    quick_ratio   = ((ca - inv) / cl) if pd.notna(ca) and pd.notna(cl) and cl != 0 else pd.NA
    debt_ratio    = (tl / eq) if pd.notna(tl) and pd.notna(eq) and eq != 0 else pd.NA

    print(f"\n📅 latest BS date: {latest_idx}")
    print("  CA:", ca, " CL:", cl, " INV:", inv, " TL:", tl, " EQ:", eq)
    print("  → current_ratio:", current_ratio)
    print("  → quick_ratio:  ", quick_ratio)
    print("  → debt_ratio:   ", debt_ratio)
else:
    print("\n⚠️ 유동자산/유동부채 컬럼을 못 찾았습니다. 위 MAPPED COLS가 None인 항목의 실제 컬럼명을 aliases에 추가하세요.")


✔ corp: 현대자동차 005380                                        
✔ bs_flat shape: (13, 58)
✔ bs_flat latest index: 20241231
🔎 MAPPED COLS: {'CA': '유동자산', 'CL': '유동부채', 'TL': '부채총계', 'EQ': '자본총계', 'INV': '재고자산'}
🔎 BS columns (first 25): ['유동자산', '현금및현금성자산', '단기금융상품', '기타금융자산', '매출채권', '기타채권', '재고자산', '기타자산', '당기법인세자산', '금융업채권', '매각예정비유동자산', '비유동자산', '장기금융상품', '기타금융자산', '장기성매출채권', '기타채권', '기타자산', '유형자산', '투자부동산', '무형자산', '공동기업 및 관계기업투자', '순확정급여자산', '이연법인세자산', '금융업채권', '운용리스 자산']

📅 latest BS date: 20241231
  CA: 115764287000000.0  CL: 79509643000000.0  INV: 19791007000000.0  TL: 219522496000000.0  EQ: 120275933000000.0
  → current_ratio: 1.4559779497437815
  → quick_ratio:   1.2070646575535497
  → debt_ratio:    1.8251572906110818


In [5]:
# ⬇ 종목코드만 바꿔서 실행
code = "005380"

from kiwoom_finance.dart_client import init_dart, find_corp, extract_fs
from kiwoom_finance.preprocess import preprocess_all
from kiwoom_finance.metrics import compute_metrics_df_flat_kor

init_dart()  # .env 로드
corp = find_corp(code)
fs = extract_fs(corp, bgn_de="20170101", report_tp="annual", separate=False)
bs_flat, is_flat, cis_flat, cf_flat = preprocess_all(fs)

print("== BS columns (first 40) ==")
print(bs_flat.columns.tolist()[:40])

# 1) 현재 metrics 계산 결과 한 번 확인
res = compute_metrics_df_flat_kor(bs_flat, is_flat, cis_flat, cf_flat)
latest = res.index.max()
print("\n== Latest date ==", latest)
print(res.loc[latest, ["current_ratio","quick_ratio","debt_ratio","equity_ratio"]])

# 2) 내부 원천값 직접 체크: 유동자산/유동부채/재고/자산총계/부채총계/자본총계 후보
CA_CANDS  = ["유동자산","유동자산총계","유동자산 총계","유동자산총액","유동자산 합계","유동자산계"]
CL_CANDS  = ["유동부채","유동부채총계","유동부채 총계","유동부채총액","유동부채 합계","유동부채계","단기부채","단기부채총계","단기부채 총계"]
TA_CANDS  = ["자산총계","자산 총계","자산총액","자산 합계","자산계","부채와자본총계","부채와 자본 총계"]
TL_CANDS  = ["부채총계","부채 총계","부채총액","부채 합계","부채계"]
EQ_CANDS  = ["자본총계","자본 총계","자본총액","자본 합계","자본계"]
INV_CANDS = ["재고자산","상품","제품","반제품","원재료","재공품","저장품"]

def pick_first(df, cands):
    for c in cands:
        if c in df.columns:
            return c
    return None

ca_col  = pick_first(bs_flat, CA_CANDS)
cl_col  = pick_first(bs_flat, CL_CANDS)
ta_col  = pick_first(bs_flat, TA_CANDS)
tl_col  = pick_first(bs_flat, TL_CANDS)
eq_col  = pick_first(bs_flat, EQ_CANDS)
inv_col = pick_first(bs_flat, INV_CANDS)

print("\n== MAPPED COLS ==")
print({"CA": ca_col, "CL": cl_col, "TA": ta_col, "TL": tl_col, "EQ": eq_col, "INV": inv_col})

import pandas as pd

def to_num(x): 
    return pd.to_numeric(str(x).replace(",",""), errors="coerce")

latest_idx = int(sorted(bs_flat.index)[-1])
row = bs_flat.loc[latest_idx]

CA  = to_num(row.get(ca_col))  if ca_col  else pd.NA
CL  = to_num(row.get(cl_col))  if cl_col  else pd.NA
TA  = to_num(row.get(ta_col))  if ta_col  else pd.NA
TL  = to_num(row.get(tl_col))  if tl_col  else pd.NA
EQ  = to_num(row.get(eq_col))  if eq_col  else (TA - TL if (pd.notna(TA) and pd.notna(TL)) else pd.NA)
INV = to_num(row.get(inv_col)) if inv_col else 0

CR  = (CA / CL) if (pd.notna(CA) and pd.notna(CL) and CL != 0) else pd.NA
QR  = ((CA - INV) / CL) if (pd.notna(CA) and pd.notna(CL) and CL != 0) else pd.NA
DR  = (TL / EQ) if (pd.notna(TL) and pd.notna(EQ) and EQ != 0) else pd.NA
ER  = (EQ / TA) if (pd.notna(EQ) and pd.notna(TA) and TA != 0) else pd.NA

print("\n== Raw values on latest BS date ==")
print(f"CA={CA}, CL={CL}, INV={INV}, TA={TA}, TL={TL}, EQ={EQ}")
print(f"→ current_ratio={CR}, quick_ratio={QR}, debt_ratio={DR}, equity_ratio={ER}")


KeyboardInterrupt: 

In [6]:
from importlib import reload
from kiwoom_finance import metrics

reload(metrics)
from kiwoom_finance.metrics import compute_metrics_df_flat_kor

from kiwoom_finance.dart_client import init_dart, find_corp, extract_fs
from kiwoom_finance.preprocess import preprocess_all

init_dart()
corp = find_corp("005380")
fs = extract_fs(corp, bgn_de="20170101", report_tp="annual", separate=False)
bs_flat, is_flat, cis_flat, cf_flat = preprocess_all(fs)

df, dbg = compute_metrics_df_flat_kor(
    bs_flat, is_flat, cis_flat, cf_flat, return_debug=True
)

print("== 선택된 컬럼명 ==", dbg["chosen_cols"])
print("== 최신키 ==", dbg["latest_key"])
print("== 최신 원시값 ==", dbg["latest_raw_values"])
print("== 최신 비율(숫자) ==", dbg["latest_ratios_numeric"])


== 선택된 컬럼명 == {'CA': '유동자산', 'CL': '유동부채', 'TA': '자산총계', 'TL': '부채총계', 'EQ': '자본총계', 'INV': None}
== 최신키 == 20241231
== 최신 원시값 == {'CA': 115764287000000.0, 'CL': 79509643000000.0, 'INV': 19791007000000.0, 'TA': 339798429000000.0, 'TL': 219522496000000.0, 'EQ': 120275933000000.0, 'R': 175231153000000.0, 'OI': 14239592000000.0, 'FC': 898551000000.0}
== 최신 비율(숫자) == {'current_ratio': 1.4559779497437815, 'quick_ratio': 1.2070646575535497, 'debt_ratio': 1.8251572906110818, 'equity_ratio': 0.35396259292299437}


In [3]:
from kiwoom_finance.dart_client import init_dart, find_corp, extract_fs
from kiwoom_finance.preprocess import preprocess_all
from kiwoom_finance.metrics import compute_metrics_df_flat_kor

init_dart()  # 또는 init_dart(api_key="...")

corp = find_corp("005380")
fs = extract_fs(corp, bgn_de="20170101", report_tp="annual", separate=False)
bs_flat, is_flat, cis_flat, cf_flat = preprocess_all(fs)

df, dbg = compute_metrics_df_flat_kor(bs_flat, is_flat, cis_flat, cf_flat, return_debug=True)
print(dbg["chosen_cols"])
print(dbg["latest_raw_values"])
print(dbg["latest_ratios_numeric"])
print(df.tail(1))

Error occurred during getting browser(s): random, but was suppressed with fallback.


{'CA_alias_candidates': ['유동자산'], 'CL_alias_candidates': ['유동부채'], 'TA_alias_candidates': ['자산총계', '부채와자본총계'], 'TL_alias_candidates': ['부채총계'], 'EQ_alias_candidates': ['자본총계'], 'CA_relaxed': [], 'CL_relaxed': [], 'TA_relaxed': [], 'TL_relaxed': [], 'EQ_relaxed': [], 'INVENTORY_cols': ['재고자산'], 'AR_cols': ['매출채권', '기타채권', '장기성매출채권', '기타채권'], 'BORROWINGS_cols': ['단기차입금', '유동성장기부채', '기타금융부채', '리스부채', '장기성미지급금', '사채', '장기차입금', '기타금융부채', '리스부채']}
{'CA': np.float64(115764287000000.0), 'CL': np.float64(79509643000000.0), 'TA': np.float64(339798429000000.0), 'TL': np.float64(219522496000000.0), 'EQ': np.float64(120275933000000.0), 'INV': np.float64(19791007000000.0), 'AR': np.float64(17229088000000.0), 'OI': np.float64(14239592000000.0), 'R': np.float64(175231153000000.0), 'FC': np.float64(898551000000.0), 'EBITDA': nan, 'CFO': np.float64(-38801112000000.0), 'CapexOutflow': np.float64(10367455000000.0), 'NetIncome': np.float64(25756599000000.0)}
{'current_ratio': np.float64(1.4559779497437815)

In [4]:
# debug_check.py (또는 노트북 셀)
from kiwoom_finance.dart_client import init_dart, find_corp, extract_fs
from kiwoom_finance.preprocess import preprocess_all
from kiwoom_finance.metrics import compute_metrics_df_flat_kor

init_dart()  # .env의 DART_API_KEY 사용

corp = find_corp("005380")  # 원하는 종목코드
fs = extract_fs(corp, bgn_de="20170101", report_tp="annual", separate=False)
bs_flat, is_flat, cis_flat, cf_flat = preprocess_all(fs)

df, dbg = compute_metrics_df_flat_kor(
    bs_flat_df=bs_flat,
    is_flat_df=is_flat,
    cis_flat_df=cis_flat,
    cf_flat_df=cf_flat,
    return_debug=True,   # ← 여기!
)

print("== 선택된 컬럼명 ==", dbg["chosen_cols"])
print("== 최신키 ==", dbg["latest_key"])
print("== 최신 원시값 ==", dbg["latest_raw_values"])
print("== 최신 비율(숫자) ==", dbg["latest_ratios_numeric"])
print(df.tail(1))  # 최신 행 확인


== 선택된 컬럼명 == {'CA_alias_candidates': ['유동자산'], 'CL_alias_candidates': ['유동부채'], 'TA_alias_candidates': ['자산총계', '부채와자본총계'], 'TL_alias_candidates': ['부채총계'], 'EQ_alias_candidates': ['자본총계'], 'CA_relaxed': [], 'CL_relaxed': [], 'TA_relaxed': [], 'TL_relaxed': [], 'EQ_relaxed': [], 'INVENTORY_cols': ['재고자산'], 'AR_cols': ['매출채권', '기타채권', '장기성매출채권', '기타채권'], 'BORROWINGS_cols': ['단기차입금', '유동성장기부채', '기타금융부채', '리스부채', '장기성미지급금', '사채', '장기차입금', '기타금융부채', '리스부채']}
== 최신키 == 20241231
== 최신 원시값 == {'CA': np.float64(115764287000000.0), 'CL': np.float64(79509643000000.0), 'TA': np.float64(339798429000000.0), 'TL': np.float64(219522496000000.0), 'EQ': np.float64(120275933000000.0), 'INV': np.float64(19791007000000.0), 'AR': np.float64(17229088000000.0), 'OI': np.float64(14239592000000.0), 'R': np.float64(175231153000000.0), 'FC': np.float64(898551000000.0), 'EBITDA': nan, 'CFO': np.float64(-38801112000000.0), 'CapexOutflow': np.float64(10367455000000.0), 'NetIncome': np.float64(25756599000000.0)}
==

In [6]:
df, dbg = compute_metrics_df_flat_kor(bs_flat, is_flat, cis_flat, cf_flat, return_debug=True)

latest_key = dbg["latest_key"]
print("latest_key:", latest_key)

# 최신 연도 1행만 보기
print(df.loc[[latest_key]])

latest_key: 20241231
                 debt_ratio        equity_ratio  debt_dependency_ratio  \
date                                                                     
20241231 1.8251572906110818 0.35396259292299437    0.47141534900975074   

              current_ratio        quick_ratio  interest_coverage_ratio  \
date                                                                      
20241231 1.4559779497437815 1.2070646575535497       15.847283014542302   

         ebitda_to_total_debt    cfo_to_total_debt        free_cash_flow  \
date                                                                       
20241231                 None -0.17675232701435756 -49,168,567,000,000.0   

            operating_margin                roa                 roe  \
date                                                                  
20241231 0.08126176057290452 0.0827838047017202 0.23195223217154423   

           net_profit_margin  total_asset_turnover  \
date                             

In [ ]:
from kiwoom_finance.batch import get_metrics_for_codes
from kiwoom_finance.dart_client import find_corp, extract_fs
from kiwoom_finance.preprocess import preprocess_all
from kiwoom_finance.metrics import compute_metrics_df_flat_kor

# 1) 한 종목 선택 (문제가 생기는 종목으로 바꿔서 반복)
corp = find_corp("005380")  # 예: 현대차
fs = extract_fs(corp, bgn_de="20170101", report_tp="annual", separate=False)
bs_flat, is_flat, cis_flat, cf_flat = preprocess_all(fs)

# 2) 디버그 켜서 계산
df, dbg = compute_metrics_df_flat_kor(bs_flat, is_flat, cis_flat, cf_flat, return_debug=True)

print("== 선택된 컬럼명 ==", dbg["chosen_cols"])
print("== 최신키 ==", dbg["latest_key"])
print("== 최신 원시값 ==", dbg["latest_raw_values"])
print("== 최신 비율(숫자) ==", dbg["latest_ratios_numeric"])

In [5]:
from kiwoom_finance.aliases import KOR_KEY_ALIASES
# 예시: 후보에서 골라서 추가 (여기 문자열을 네 출력값으로 교체!)
KOR_KEY_ALIASES["revenue"] += ["영업수익"]
KOR_KEY_ALIASES["operating_income"] += ["영업이익(손실)"]
KOR_KEY_ALIASES["net_income"] += [
    "지배기업의 소유주에게 귀속되는 당기순이익",
]
KOR_KEY_ALIASES["finance_costs"] += ["이자비용"]

from kiwoom_finance.metrics import compute_metrics_df_flat_kor
df, dbg = compute_metrics_df_flat_kor(bs_flat, is_flat, cis_flat, cf_flat, return_debug=True)
print("latest:", dbg["latest_key"])
print(dbg["latest_raw_values"])  # NetIncome / R / OI가 숫자인지 확인
print(df.loc[dbg["latest_key"], ["operating_margin","roa","roe","net_profit_margin"]])


latest: 20241231
{'CA': np.float64(329901419756.0), 'CL': np.float64(647142868725.0), 'TA': np.float64(885079753443.0), 'TL': np.float64(715139179755.0), 'EQ': np.float64(169940573688.0), 'INV': np.float64(176193749567.0), 'AR': np.float64(96733239648.0), 'OI': nan, 'R': np.float64(764435874065.0), 'FC': np.float64(75416213776.0), 'EBITDA': nan, 'CFO': np.float64(76183274441.0), 'CapexOutflow': np.float64(8416454013.0), 'NetIncome': nan}
operating_margin     None
roa                  None
roe                  None
net_profit_margin    None
Name: 20241231, dtype: object


In [6]:
bs_flat

"([D210000] 재무상태표, 유동/비유동법 - 연결 | Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW), label_ko)",유동자산,현금및현금성자산,단기금융상품,단기대여금,매출채권및기타채권,재고자산,당기법인세자산,기타금융자산,기타자산,비유동자산,...,부채총계,지배기업 소유주 지분,자본금,자본잉여금,기타자본,기타포괄손익누계액,이익잉여금,비지배지분,자본총계,자본과부채총계
20241231,"329,901,419,756.0","40,296,836,459.0","5,989,957,174.0",0.0,"96,733,239,648.0","176,193,749,567.0","122,881,060.0","84,470,267.0","10,480,285,581.0","555,178,333,687.0",...,"715,139,179,755.0","169,940,239,646.0","10,937,873,500.0","92,662,328,851.0","9,278,965,178.0","133,837,305,181.0","-76,776,233,064.0","334,042.0","169,940,573,688.0","885,079,753,443.0"
20231231,"365,999,633,369.0","32,666,647,187.0","6,995,676,059.0","395,496,066.0","90,173,363,098.0","223,197,298,930.0","535,129.0",0.0,"12,570,616,900.0","610,559,484,076.0",...,"695,207,306,418.0","280,697,296,299.0","10,937,873,500.0","92,662,328,851.0","9,278,965,178.0","124,535,365,798.0","43,282,762,972.0","654,514,728.0","281,351,811,027.0","976,559,117,445.0"
20221231,"369,265,021,316.0","26,106,689,771.0","9,335,988,179.0","5,000,000,000.0","103,518,741,233.0","214,020,291,024.0","40,683,044.0",0.0,"11,242,628,065.0","493,323,692,687.0",...,"667,856,104,059.0","194,466,630,676.0","10,931,634,000.0","89,759,098,981.0","9,278,965,178.0","4,562,781,009.0","79,934,151,508.0","265,979,268.0","194,732,609,944.0","862,588,714,003.0"
20211231,"403,848,737,910.0","49,704,007,316.0","6,692,082,505.0",NaN,"87,448,544,773.0","250,900,631,969.0","38,541,547.0","174,906,698.0","8,890,023,102.0","491,115,331,239.0",...,"632,933,749,646.0","261,979,815,534.0","10,931,634,000.0","89,759,098,981.0","9,278,965,178.0","-372,978,648.0","152,383,096,023.0","50,503,969.0","262,030,319,503.0","894,964,069,149.0"
20201231,"333,232,900,296.0","56,753,327,974.0","14,199,869,962.0",NaN,"82,133,053,830.0","168,250,545,842.0","38,790,268.0",0.0,"11,857,312,420.0","510,903,226,308.0",...,"529,391,928,007.0","314,368,682,253.0","10,931,634,000.0","88,424,595,228.0","9,278,965,178.0","-4,223,020,326.0","209,956,508,173.0","375,516,344.0","314,744,198,597.0","844,136,126,604.0"
20191231,"344,438,124,049.0","73,660,186,177.0","9,411,080,000.0",NaN,"98,296,181,205.0","150,678,357,509.0","50,620,112.0","158,163,955.0","12,183,535,091.0","506,643,036,264.0",...,"526,810,418,216.0","323,735,928,349.0","10,931,634,000.0","88,378,008,429.0","9,278,965,178.0","-5,896,332,955.0","221,043,653,697.0","534,813,748.0","324,270,742,097.0","851,081,160,313.0"
20181231,"302,450,962,322.0","32,727,257,129.0","5,273,490,512.0",NaN,"93,786,722,212.0","156,943,685,282.0","57,217,556.0","1,717,394,232.0","11,937,325,399.0","402,804,055,308.0",...,"384,652,006,040.0","320,266,437,746.0","10,931,634,000.0","88,378,008,429.0","9,278,965,178.0","-5,069,279,156.0","216,747,109,295.0","336,573,844.0","320,603,011,590.0","705,255,017,630.0"
20171231,"358,662,119,603.0","98,436,111,567.0","1,981,900,000.0",NaN,"91,076,773,504.0","158,965,659,077.0","64,936,630.0",0.0,"8,123,508,825.0","316,934,587,405.0",...,"351,935,280,017.0","323,201,793,877.0","10,931,634,000.0","88,378,008,429.0","13,635,027,937.0","-2,075,275,219.0","212,332,398,730.0","459,633,114.0","323,661,426,991.0","675,596,707,008.0"
20161231,"344,843,073,462.0","76,932,423,528.0","5,901,353,260.0",NaN,"118,813,768,076.0","133,573,947,257.0","121,304,397.0","3,069,735,950.0","6,092,475,994.0","328,398,858,692.0",...,"368,639,652,706.0","304,602,279,448.0","10,823,400,000.0","88,378,008,429.0","13,635,027,937.0","131,511,068.0","191,634,332,014.0",NaN,"304,602,279,448.0","673,241,932,154.0"
20151231,"311,527,520,602.0","22,016,043,621.0","33,607,432,000.0",NaN,"106,913,529,627.0","141,254,436,532.0",NaN,"1,172,000,000.0","6,260,693,189.0","343,963,620,552.0",...,"466,506,513,916.0","188,984,627,238.0","1,034,050,000.0","21,377,630,726.0","18,439,911,561.0","-1,321,876,848.0","149,454,911,799.0",NaN,"188,984,627,238.0","655,491,141,154.0"


In [7]:
print("latest:", dbg["latest_key"])
print(dbg["latest_raw_values"])  # R / OI / NetIncome 숫자 나오는지
print(df.loc[dbg["latest_key"], ["operating_margin","roa","roe","net_profit_margin"]])


latest: 20241231
{'CA': np.float64(329901419756.0), 'CL': np.float64(647142868725.0), 'TA': np.float64(885079753443.0), 'TL': np.float64(715139179755.0), 'EQ': np.float64(169940573688.0), 'INV': np.float64(176193749567.0), 'AR': np.float64(96733239648.0), 'OI': nan, 'R': np.float64(764435874065.0), 'FC': np.float64(75416213776.0), 'EBITDA': nan, 'CFO': np.float64(76183274441.0), 'CapexOutflow': np.float64(8416454013.0), 'NetIncome': nan}
operating_margin     None
roa                  None
roe                  None
net_profit_margin    None
Name: 20241231, dtype: object


In [8]:
df2, dbg2 = compute_metrics_df_flat_kor(bs_flat, is_flat, cis_flat, cf_flat, return_debug=True)
print(dbg2.get("chosen_cols", {}))

{'CA_alias_candidates': ['유동자산'], 'CL_alias_candidates': ['유동부채'], 'TA_alias_candidates': ['자산총계'], 'TL_alias_candidates': ['부채총계'], 'EQ_alias_candidates': ['자본총계', '자본과부채총계'], 'CA_relaxed': [], 'CL_relaxed': [], 'TA_relaxed': [], 'TL_relaxed': [], 'EQ_relaxed': [], 'INVENTORY_cols': ['재고자산'], 'AR_cols': ['매출채권및기타채권'], 'BORROWINGS_cols': ['단기차입금', '기타금융부채', '기타 유동부채', '장기차입금', '기타금융부채']}


In [10]:
print("== IS columns ==")
print(is_flat.columns.tolist())

print("\n== CIS columns ==")
print(cis_flat.columns.tolist())


== IS columns ==


AttributeError: 'NoneType' object has no attribute 'columns'

In [14]:
from kiwoom_finance.dart_client import init_dart, find_corp, extract_fs
from kiwoom_finance.preprocess import preprocess_all
from kiwoom_finance.metrics import compute_metrics_df_flat_kor

init_dart()
corp = find_corp("143210")  # 원하는 종목
fs   = extract_fs(corp, bgn_de="20170101", report_tp="annual", separate=False)
bs_flat, is_flat, cis_flat, cf_flat = preprocess_all(fs)

# 💡 IS가 없다면 is_flat=None으로 넘겨도 됨
df, dbg = compute_metrics_df_flat_kor(
    bs_flat_df=bs_flat,
    is_flat_df=None,        # ← IS 없이
    cis_flat_df=cis_flat,   # ← CIS만으로
    cf_flat_df=cf_flat,
    return_debug=True,
)

print("latest:", dbg["latest_key"])
print(df.loc[dbg["latest_key"], ["operating_margin","roa","roe","net_profit_margin"]])

latest: 20241231                      
operating_margin     None
roa                  None
roe                  None
net_profit_margin    None
Name: 20241231, dtype: object


In [3]:
from kiwoom_finance.dart_client import init_dart, find_corp, extract_fs
from kiwoom_finance.preprocess import preprocess_all
from kiwoom_finance.metrics import compute_metrics_df_flat_kor

init_dart()
corp = find_corp("143210")  # ROA/ROE가 NaN이었던 종목
fs = extract_fs(corp, bgn_de="20170101", report_tp="annual", separate=False)
bs_flat, is_flat, cis_flat, cf_flat = preprocess_all(fs)

df, dbg = compute_metrics_df_flat_kor(bs_flat, None, cis_flat, cf_flat, return_debug=True)
print("latest_key:", dbg["latest_key"])
print("latest raw:", dbg["latest_raw_values"])  # NetIncome이 숫자인지 확인
print(df.loc[dbg["latest_key"], ["net_profit_margin","roa","roe","operating_margin"]])
print("notes:", dbg.get("notes"))


latest_key: 20241231                  
latest raw: {'CA': np.float64(329901419756.0), 'CL': np.float64(647142868725.0), 'TA': np.float64(885079753443.0), 'TL': np.float64(715139179755.0), 'EQ': np.float64(169940573688.0), 'INV': np.float64(176193749567.0), 'AR': np.float64(96733239648.0), 'OI': nan, 'R': np.float64(764435874065.0), 'FC': np.float64(75416213776.0), 'EBITDA': nan, 'CFO': np.float64(76183274441.0), 'CapexOutflow': np.float64(8416454013.0), 'NetIncome': np.float64(-111384428816.0)}
net_profit_margin   -0.14570800847387885
roa                 -0.11966276656317316
roe                 -0.49362423381613874
operating_margin                    None
Name: 20241231, dtype: object
notes: ['net_income_fallback_to_TCI']


In [6]:
!pip install joblib


  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)


In [10]:
!pip install xgboost

  Using cached xgboost-3.1.0-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-3.1.0-py3-none-win_amd64.whl (72.0 MB)


In [8]:
import sys
!{sys.executable} -m pip install scikit-learn joblib

  Using cached scikit_learn-1.7.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.16.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.2-cp311-cp311-win_amd64.whl (8.9 MB)
Using cached scipy-1.16.2-cp311-cp311-win_amd64.whl (38.7 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- -------------------------- 1/3 [scipy]
   ------------- ------------------

In [13]:
from train_xgb_credit import main, argparse

args = argparse.Namespace(
    excel_path=r"C:\Users\edukd\OneDrive\문서\GitHub\kiwoom\stock.xlsx",
    target_col="credit_ratings",   # ← 여기만 변경
    out_dir="model_out",
    valid_ratio=0.2,
    n_estimators=1000,
    max_depth=6,
    learning_rate=0.05,
    early_stopping_rounds=50,
)
main(args)


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.